In [1]:
%load_ext dotenv
%dotenv

In [2]:
import lyricsgenius
genius = lyricsgenius.Genius()
genius.verbose = False
genius.remove_section_headers = True
genius.skip_non_songs = True

In [3]:
from tqdm.auto import tqdm
tqdm.pandas()

In [4]:
import sqlite3
import os
import pickle

In [5]:
SQL_FILEPATH = os.getenv("PLAYBACK_FILE")
print(SQL_FILEPATH)
lyrics_db = sqlite3.connect(SQL_FILEPATH)

22-04-29-playback-fm-top-pop.db


In [6]:
# Load Data

In [7]:
import pandas as pd
df = pd.read_sql_query("SELECT * FROM tracks", lyrics_db)
print(df.head())

   index  year           artist                        track  rank  \
0      0  1985   USA For Africa             We Are the World     1   
1      1  1985             A-Ha                   Take On Me     2   
2      2  1985        Foreigner  I Want to Know What Love Is     3   
3      3  1985  Tears For Fears                        Shout     4   
4      4  1985    Lionel Richie              Say You, Say Me     5   

                                                link  
0  /charts/top-100-songs/video/1985/USA-For-Afric...  
1    /charts/top-100-songs/video/1985/AHa-Take-On-Me  
2  /charts/top-100-songs/video/1985/Foreigner-I-W...  
3  /charts/top-100-songs/video/1985/Tears-For-Fea...  
4  /charts/top-100-songs/video/1985/Lionel-Richie...  


## Create Unique Songs Dataframe

In [8]:
df.rename(columns = {'index':'track_id'}, inplace=True)

In [9]:
print(df.head())

   track_id  year           artist                        track  rank  \
0         0  1985   USA For Africa             We Are the World     1   
1         1  1985             A-Ha                   Take On Me     2   
2         2  1985        Foreigner  I Want to Know What Love Is     3   
3         3  1985  Tears For Fears                        Shout     4   
4         4  1985    Lionel Richie              Say You, Say Me     5   

                                                link  
0  /charts/top-100-songs/video/1985/USA-For-Afric...  
1    /charts/top-100-songs/video/1985/AHa-Take-On-Me  
2  /charts/top-100-songs/video/1985/Foreigner-I-W...  
3  /charts/top-100-songs/video/1985/Tears-For-Fea...  
4  /charts/top-100-songs/video/1985/Lionel-Richie...  


## Find Lyrics

In [10]:
def find_lyrics(row):
    try:
        return genius.search_song(row["track"], row["artist"])
    except:
        return "Error"

In [11]:
df["lyrics"] = df.progress_apply(find_lyrics, axis=1)
df.to_pickle("22-04-21-lyrics-pickle")

  0%|          | 0/3595 [00:00<?, ?it/s]

In [12]:
# df.to_sql("tracks2", lyrics_db)

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3595 entries, 0 to 3594
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   track_id  3595 non-null   int64 
 1   year      3595 non-null   int64 
 2   artist    3595 non-null   object
 3   track     3595 non-null   object
 4   rank      3595 non-null   int64 
 5   link      3595 non-null   object
 6   lyrics    3555 non-null   object
dtypes: int64(3), object(4)
memory usage: 196.7+ KB


In [14]:
df = df.rename({"lyrics": "lyrics_object"}, axis=1)
df["lyrics"] = df.progress_apply(
    lambda row: row["lyrics_object"].lyrics if row["lyrics_object"] and not isinstance(row["lyrics_object"],
                                                                                       str) else None, axis=1)

  0%|          | 0/3595 [00:00<?, ?it/s]

In [1]:
df.drop("lyrics_object", axis=1).to_sql("tracks", lyrics_db, if_exists="replace")

NameError: name 'df' is not defined